In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext line_profiler
%matplotlib inline
from tqdm import tnrange, tqdm_notebook

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
import sys

import re

In [2]:
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)

In [3]:
!ls -alF ../input

total 2283108
drwxrwxr-x 2 chrisyang chrisyang       4096 12월 29 21:20 ./
drwxrwxr-x 9 chrisyang chrisyang       4096 12월 29 23:19 ../
-rw-rw-r-- 1 chrisyang chrisyang 1737181862 12월 28 13:19 fasttext_200.bin
-rw-rw-r-- 1 chrisyang chrisyang   74451286 12월 29 21:20 fasttext_vocab.pkl
-rw-rw-r-- 1 chrisyang chrisyang    3576801 12월 29 21:19 preprocessed_test.csv
-rw-rw-r-- 1 chrisyang chrisyang  394642565 12월 29 21:19 preprocessed_train.csv
-rw-rw-r-- 1 chrisyang chrisyang    1167417 12월 16 13:15 public_test.csv
-rw-rw-rw- 1 chrisyang chrisyang  126845824 11월 28 22:04 train.csv


In [5]:
DATASET_PATH = '../input'

train_df = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATASET_PATH, 'public_test.csv'))
preprocessed_train_df = pd.read_csv(os.path.join(DATASET_PATH, 'preprocessed_train.csv'))
preprocessed_test_df = pd.read_csv(os.path.join(DATASET_PATH, 'preprocessed_test.csv'))

In [6]:
train_df.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


In [7]:
test_df.head()

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...


In [8]:
preprocessed_train_df.head()

,id,year_month,text,smishing,jamo
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,ㅇㅡㄴㅎㅐㅇㅅㅓㅇㅅㅏㄴ ㅌㅣㅁㅈㅏㅇㅇㅣㅂㄴㅣㄷㅏ ㅎㅐㅇㅂㅗㄱㅎㅏㄴㅈㅜㅁㅏㄹㄷㅚㅅㅔㅇㅛ
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,ㅇㅗㄴㅡㄹㄷㅗㅁㅏㄶㅇㅣㅇㅜㅅㅇㅡㅅㅣㄴㅡㄴㅎㅏㄹㅜㅅㅣㅈㅏㄱㅎㅏㅅㅔㅇㅛ ㅇㅡㄴㅎㅐㅇ ㅈ...
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ ㄱㅗㄱㅐㄱㄴㅣㅁ ㅇㅡㄴㅎㅐㅇㅇㅣㅂㄴㅣㄷㅏ ㄱㅡㅁㅇㅣㄹ ...
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,ㄱㅗㄱㅐㄱㄴㅣㅁㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ ㅇㅡㄴㅎㅐㅇ ㅈㅣㅈㅓㅁㅇㅣㅂㄴㅣㄷㅏㅈㅣㄴㅏㄴ...
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,ㅇㅝㄹㅇㅡㄴ ㅅㅐㄹㅗㅇㅜㅁㅇㅣ ㄱㅏㄷㅡㄱ ㅇㅣㅂㄴㅣㄷㅏ ㅇㅗㄹ ㅎㅏㄴㅎㅐ ㄷㅓ ㅁ...


In [11]:
preprocessed_test_df.head()

,id,year_month,text,jamo
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...,ㄱㅗㄱㅐㄱㄴㅣㅁ ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ ㅅㅐㄹㅗㅂㄱㅔ ㅅㅣㅈㅏㄱㅎㅏㄴㅡㄴ ㅎㅏㄴ ...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...,ㄱㅣㄴㄱㅡㅂ ㅇㅏㄴㄴㅐ ㅇㅡㄴㅎㅐㅇ ㄱㅏㄹㅏㄱㄷㅗㅇ ㅈㅣㅈㅓㅁ ㅎㅔㄹㄹㅣㅇㅗ ㄱㅣ...
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...,ㄱㅗㄱㅐㄱㄴㅣㅁ ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛㅇㅗㄹㅎㅐㄴㅡㄴ ㅁㅣㅅㅔㅁㅓㄴㅈㅣㄱㅏ ㅇㅠㄴ...
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...,ㄱㅗㄱㅐㄱㄴㅣㅁㅊㅏㅈㅇㅏㅇㅗㄴ ㅎㅐㅇㅇㅜㄴㅇㅡㄹ ㅈㅏㅂㅇㅡㅅㅕㅆㄴㅏㅇㅛ ㅁㅗㅅㅈㅏ...
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...,ㄱㅗㄱㅐㄱㄴㅣㅁㅅㅐㅎㅐ ㅂㅗㄱ ㅁㅏㄶㅇㅣ ㅂㅏㄷㅇㅡㅅㅔㅇㅛ ㅇㅡㄴㅎㅐㅇ ㅋㅗㅅㅡㅌ...


In [12]:
import pickle
FASTTEXT_PATH = os.path.join(DATASET_PATH, 'fasttext_vocab.pkl')
with open(FASTTEXT_PATH, 'rb') as f:    
    fasttext_vocab = pickle.load(f)


In [16]:
for key, value in fasttext_vocab.items():
    print(key, end = ' ')

X </s> X올림 X- X은행 수 고객님 행복한 감사합니다. 바랍니다. 저희 있습니다. : . 및 - 3. 항상 2. 하루 고객님의 주셔서 드립니다. 따라 있는 주말 최대 주시면 꼭 감사드립니다. 즐거운 보내세요 경우 올림 더 좋은 부탁드립니다. 한 1. 4. 오늘도 X무료수신거부 X지점 X입니다. 많이 합니다. 또는 상담 고객님께 상품 최선을 답장을 가득한 2017-04_ 매우 되세요 대한 등 소중한 바로 거래해 되시길 진심으로 전화를 감사드리며 2017-09_ 함께 고객님께서 있으시면 통해 대출 가능 언제든지 감사합니다 2017-06_ 거래 잘 가능합니다. 대 도움이 늘 고객만족도 후 바랍니다 다양한 마무리 X은행을 연 오늘 모두 전화 2017-05_ 이용해주셔서 많은 최저 더욱 바라며 거래해주셔서 없이 대해 심의필 X팀장 시간 X점 한도 최고 연락 즐겁고 입니다. 관련 드리겠습니다. 안내 기원합니다. 높은 때 큰 모든 됩니다. 이용해 건강 직장인 보내세요. X) 고객님! 감사 친절히 내 행복이 편안한 최 위한 5. 다시 궁금하신 본 신용등급 고객님을 이라고 있도록 위해 찾아주셔서 인해 월 새로운 업무처리 안내드립니다. 2018-03_ 다하겠습니다. 지점 작으나 2017-11_ 가득 상담으로 있다면 X지점을 앞으로도 하는 그 현재 2017. 6. 수신을 따뜻한 않도록 마시고 상담해 분. 원리금 방식으로 일이 가능. 하시는 매우동의 원하시는 자유롭게 X계장 혹시 추가 금리를 혜택 상담이 해 드림 X대리 점은 2018-09_ 등급에 주시거나 하겠습니다. ( 팀장 안전한 2018-05_ 받으시면 이자만 잦은 하루도 지점을 도와드리겠습니다. 제 으로 기존 할 시 건수가 자체 2018. 가득하시길 일 서비스로 상관없이 X드림 주신다면 문자로 2018-04_ 2017-10_ 중 다해 전담직원 안녕하십니까? 추가진행이 보내시길 안녕하세요 이번 혜택이 다운로드 최근 X은 상담이라고 주 12. 마음으로 사용이 한번에 차등적용. 관리가 기분 전화주시면 문자를 유의하시고 드

 가능유무 환하게 제공하고 방문하셔야 glUPkjQv 나 받으셨거나 발생이 좋게 바 유리한 고객님업무처리를 모두가 2018-02_(광고) 전용페이지에서 비교해 거래감사드리며 포토폴리어를 마세요! X이나 없으실 바로가기안드로이드: 저금리- 끝자락을 타 부재중일시 달달한 평생 설문조사시매우동의 감사히 출을 받게 -꼭 읽어봐 고금리대출 기회를 대폭상승!!4. 어플 현금서비스나 드림. 신청하셔서생활의 출시안내본상품은 연소득대비 새희망홀씨 받으셨을텐데우선 내역은 한결 평소 저희지점 과다조회로 계절의 쿠폰 X사금융등을 상품은 영업점에 마무리하는 문을 역시 싶습니다. 6개월이상상품기간: 함께한 않아도!!공인인증서가 300%까지. 없어도!!간단히 금리부분도 주소만 든 담보문의도 광고물 행운이었으며 보유 금융정보와 활력을 안되는 X스타알림을 지식도 업무처리해드렸던 기프티콘 결실 안내입니다. 계좌 있습니다한도:연봉대비300%까지금리:최저 예약상담 X스타알림어플을 주부 X드림론)1. 원하시거나 X주임 당연히 스마일 후에는 분들중 2배까지 7천만원 고객님수요일엔 오류가 수요일 상품진행시 2018년도 대출금액별 리브(Liiv)에서 동의한다로 신용조회 화이팅 NIKKEI225 당신이 채무통합(대환상품)추가상품 300만원 무방문 1인 직장인전용 바랍니다준법감시인 드림론의 있습니다안드로이드 있으시고 처리해 X시화지점 가입해야 검색친구추가후 드시면 알아두세요 문의하시거나 갈아타셔도 행사는 머무는 주말보내세요. 주되세요 행복은 만드시고 알림서비스를 드림론금리최 이자절감 바로가기인터넷뱅킹을 긍정적인 설문시 07 진행가능- 바쁘시겠지만 X대 10%대로 내일 05. 참고바랍니다. 알지만 1% 3 쌀쌀해진 분명히 귀중한 o삶을 업무처리가 만19세이상적용금리: 6개월이상상환기간: 함께하시길 반환 사업자금 신용도가 투자정보 만들어갑니다. 만기일과 LMS 더없이 bank나의 환율정보 감사함이 금융소비자 은행여부 만족하셨는지요? 1억3천만원까지(대환자금추가자금)조건만23세이상직장인개인사업자 5년동안 되세요

 많거나 동의합니다 생각됩니다. 제한될 시까지(800억)-한도:기존 열매를 칭찬도 온다고 금융상품을 뒤 15까지) 전화( 부정적인 열려 조만간 투자해 도와드릴것을 환급) 250%까지. 말씀해 고객님2018년 유의 정부 듣고 다녀오신 X은행원종동 통장으로 X고객님께는 아니다. 가이드를 MVP. 진단 오후되세요 매도 즐거운주말보내세요 내점거래 안부를 부과 무거운 오셔서 지참)하며 가능(신분증만 확대 축소 원리금이 소식으로 판단-자서진행으로 진행가능-자체내부 거래감사드립니다. 진행가능2. 저희은행 믿어 결실을 6월! 장소 불편함은 금융사를 업무처리도와드린 최저3. X달 X일 미래를 X우만동 X수퍼정기예금에 지금부터 판단(신용등급이 즐거운주말 200% 응원합니다 읽어보세요. 비대면 해주세요 옥포지점 받으셔서 같아요. X은행철산역 전환대출한도: X은행구월동종합금융센터 유난히 필요한건 객관적으로 꿈모아 론의 축하합니다 합리적인 시기를 고객님늘 X은행홍제동 X금융 따사로운 했는데 기대합니다. 화이팅하세요 홍보성 X은행주안역 메시지로 앉아 필요시 X드려요 X은행동 나들이 X은행잠 차장입니다 11월의 물든 보답하는 홍콩H 3월이 것이라는 행복을응원합니다화이팅!! 국제유가 연준의 궁금하신점이나 고객님행복한 궁금하신점은 문자의 풍성하고 7월의 X옥천 내부등급으로 진단에 S 원클릭 하며 X광 각 완화 이야기 매우만족으로 늪에서 진행심의필2017-02-013(유효기간:2017. 진심의 대부분 전용메뉴를 대상거래 즐거운주말되세요 기해년 X은행대구이시아폴리스 의심치 이름만 만기-대출연장으로 부탁드립니다! 1차 개인사업장 웃음꽃이 15일 힘내세요 죄송합니다 오랜 처방이 확인부탁드립니다. 그리워지는 부분에도 X은행서 너무도 봄은 예금자보호법에 쌀쌀해졌습니다. 2만원 특별 스타링크 주식투자를 최대우대금리 빚테크도 차등적용)건수 기관은 휴대폰에서 되게끔 거래하시는데 -일반업체개인사업장 된다는것을 채우시는주말되세요 5월! X은행도마동 감사드립니다! 이루어진다면 될껍니다. 발표 비 30만원이상 행

봄길이 해외송금 X은행인후동 X아 부탁드립니다감사합니다 전적으로 X대덕테크노밸리 나왔습니다. 담는 영업을 2017-11_안녕하세요 고객님안녕하십니까?항상 23 하시도록 X옥포가 얼굴에 애플 상승폭을 X은행영 기억에 M 세액 연휴로 받으십시요 항셍: 마음가짐으로 안내한도: X도봉 연간납입액 추가한도 직통 이용할수잇습니다. 못해 사기유형으로 X은행양주고읍 정규직 사업개시 7 X대리- 대리입니다항상 아침입니다 그늘이 활용 X? 이용하신분은 문턱에 만족하실때까지 X옥동 이슈 맞는KICS 활동자급여소득자 분들대출금액은 모든일 사칭해 X은행수송동 2017-04_4월의 X드림론 이러한 간편하고 추위에 알아봐드립니다채무 점을 봄꽃과 건수 70 홍보차 걸리지 사람들을 털어내는 오 소망이 오곡을 X림 나누어 X은행하당종합금융센터 오지 상환금이 1800-9999) 대출전환 지혜롭게 X은행고척동 땐 실행 수탁법인상담전화 동장군의 쉽지만은 자산증식이 1억4천만원신용도와 증가 요구하면 뿐만 대상)퇴직연금 X은행밀양 업무처리로 복된 점검해보고 여유로 내용변경 응원 속도가 유의하세요. 피우는 glxpVFzE아이폰 주임입니다. VIP룸 소망을 X은행중계동 경우는 X팀원 문의하세요. 요청 당연하지만 하락한 싶다면?위사항에 물 2017-05_행복한 꽃잎은 X무료 XI 묻어나는 전화(02-6000- 준비해서 세트 소리가 일상의 금액과 우려와 서면으로 짬짬이 일로 계약 저희지점은 X은행간석동 곳에서 원달러 )또하나!! 2017-04_오늘도 내수경기 있었습니다. 한주간 X영통 수량이 성공 통합하여 Dow 11일 X주안지점 X은행논현동 꿈모아론금리최저 드립니다언제나 25 X은행의정부 좋다. 2017-11_소중한 (한도는 시작하세요! X장 사과드리며 가슴속에 이해를 담당자에게 2018-06_안녕하세요 X송내역 (5점) 봄처럼 싶으시면 너를 2억원에서 해드리는 bank금전을 우대)-한도1억4천만원까지-일반업체개인사업장 전달드립니다 445-10호 파장동지점 신협입니다. X송우 WTI(BBL): 계신다면 추

번호는 X팀장드림. 신용대출gt 만큼이나 2018-08_안녕하세요 고객님기분좋은 법인 X은행명동역 운영합니다. korea 신용상담을 보러 제천지점 하시는분6. 2018-09_고객님의 불편하시거나 보내세요감사합니다 대폭 3가지 방문하시거나 발령받은 모든일들이 하나씩 모습입니다. 태어난 구월동종합금융센터 주위를 전반적만족도 시간보내세요 현상황에 강화를 세종로지점 드립니다가조회는 송우지점 X가양동 경안종합금융센터 단풍과 말입니다. X은행훼밀리타운 2017-12_고객님의 4일 동호수 X의정 고객님!기존 봄날씨처럼 들 고객님2019년 2017-11_소중한거래 묵묵히 낙폭이 일반자금 인사드립니다 잔액 학익동 XN 발급받으신 2018-03_희망이 X은행봉은사로 리브(Liiv)를 전화받으시면매우 X미금역 X동탄다은 햇볕 X은행야탑역 넘 중간선거가 친절직원 개인과 X스타뱅킹에서 까지! 우수고객님께 X상봉역 종로3가지점 연계되어 퇴직연금(IRP) 2017-05_(광고)한국 어려움을 대출금리는 되십시오! 가볍고 X은행충주 녕하세요. 끼쳐드려 거치기간이 36개월 부족했더라도 고객님새해 같아라는 (신분증 X에서는 칭찬과 이날 이부분을 주위의 이번주말도 근무하게 마무리잘하시고 넉넉해지는 X은행황금 X은행광장동 담겼으면 정상회담을 X광화문역 꽃보다 신용보증재단 드리겠습니다은행권대환 X십 X행당동 철원지점 X특급송 없으셨나요 경우. 금리우대쿠폰을 시까지(1200억)-한도: 서늘한 X은행화성향남종합금융센터 되신다면매우동의로 대출대상 2%자격: 보지 2017-04_기쁨이 X주임- X매 상이)3. 서비스-최근 솔솔 대출취급이 X높은금리 였습니다. X현명한 내당동종합금융센터 X은행신해운대 X은행양평동종합금융센터 35 X은행서인천 X신부동지점 고객님안녕하세요!오늘 까지) 작성 즉 살아갈 2018-05_5월 잡아 환전우대 발생한 긍정적일 외출하실때 의장 상담받아새해들어 안내차전화드렸으나 가져 부탁드릴게요 X은행달성공단종합금융센터 임직원을 발급이 처리되는 상품만 2월이 X차 안녕하십니까?다름이 201

...

등)에게 동해아 소상공인에게 막판에 문화혜택을 X은행성동 사기- 회원별이용상품별 다운로드2. 고객님331일까지 비오는데 대행해 10521. 2018-05_(광고)안녕 기념공동구매정기예금판매1. 초과:연2. 4(일)3. 개인4. 6개월5. 커집니다 서운해 3일이내 X오세요 업체에 등)- 최초가격(23521원)의 제일인양활짝들 X소상공인 96%-WTI(BBL): 두눈으로 81%국내:코스피 위해집단 X팔용동직원일동 마무리하는6월의 느끼며가슴 70원-0. 가입대상자가 외환업무를 준비와 같아요항상 이순간이 종이접기를 통합하실분4. 50%최대 02pt-0. 있습니다!!내선전화가 되시길진심으로 46%-원달러:1184. 이여. 많으신분3. 2018-10_10월을 됩니다서민들의 안내서비스새롭게 감기로 약정금리는 2017-02_보람되고 뒤의 칼날로 착이다. 24일소중한예금 드려요다운 97%미국유럽등-Dow 버려라. 연소득2배최저금리 계절입니다고객님도 4만원)2. 의욕이 안되니 2017-03_안녕하십니까. 웃음에는 X과장추천 확대(최고 부터3. 말이생각납니다. 일반기업체 포근해지고 버핏의 오지도 합니다예쁜 고객님올해의 걸어야 10월입니다! 매우만족동의 대납이 신천역지점입니다. -데일 원하실때 !!2. 꽃소식이 사람내 2017-04_부탁드립니다 햇살기분좋은 노동절로 드린지 X추천당부드립니다. 2017-04_일교차가 줍니다행복은 올림벚꽃목련고운 아니었을까 반야월지점에서 시작에는 닫혀있던 유가(WTI)는 6개월계약금액 62446. 추운거 2차로 2017-04_거래감사드립니다매우동의로 10분의 부양정책으로 2107. 급락중국 주시면서 08:0024:00 16원 낙폭과대업종 등록으로 있습니다어플 68pt-0. 구월아시아드(출) 해당서류 1644-7958 2017-11_어제는 역사이고 Jones:19954. 50%-SP500: X고객님!안녕하세요? 사랑하려고 금융생활이 시작되었습니다이번 28%-Euro 좋은저녁되시길 해외바이어와 3일간(주말제외) 바쁘시더라도시간한번내셔서꼭갱신부탁드립니다. 

필요함 본인확인계좌번호변경 2)저금리 X화성팔탄지점 당일처리 신청해야 당월 분석임 가능)연금도 누려보세요!더 기분UP 다시한 드세요- 뿐이다내일은 화양연화의 드립니 순간이길 원본2. 도리인줄 토닥 나의집 조정. 아니다미래는 2018-04_고객님께 유럽과 2단지 보복조치에 한국증시가 활기찬한주되세요. 중-오랜만에 감기조심하세요!- 층 눈부시네요. 드리며. 풍요로운추석 부회장의 김포지점으로 모두에 주안역지점을 -오늘-남들은 2017-04_언제나잊지 하루되십시요! 공사관련으 개인형IRP(퇴직연금) 마음편히 2017-08_(광고)신용협 리모델링으로 산업생산이 옵션만기일을 아니다내가 혜안을 SK주유소 수놓은 X차장메가폴리스롯데마트 변경사항입니다. 33pt(-0. 품안에 해서인지 신용카드로 듬뿍받아 위한해와 걸어보셨는지요?힘차고 2017-08_과거의 X송파지점을 마저도 시간도풍성하시길 X팀장입니다어제 불행해진다면그건 주인공으로 책임이다. 2017-02_기억에 X송우종합금융센터 2018-04_각기 등록일 않는다하네요. 일정입니다. 도달해서도 돼지해 전업종 하였습니다바빠서 주세요!신규대출 아니오라2017년 잘해주시고 타행만기나 있네요가을 마음재물건강웃음행복행운 45%대 용서하는 얻는건 2018-11_황금빛 1186. 무역대금 1119일부터 해킹에 쓰셨습니다. 살피시고 (ㅡㅡㅡ)설문조사시매우동의!꼭 바야흐로 조심하시고요. 뱅킹을 한걸음씩 2018-11_내년도 보낸날은 930109까지는 보답이 활짝웃는하루되세요 멋진주말보내세요. 웃지않고 넣어주네요 길목길목 신탁상품은 하늘빛이 조급하게 늙어갑니다. 서명의 2018-11_내 언제든전화주세요 새해를맞이하게 감사드립니다o행복하고 위)전체메뉴 STEP01. 2017-11_고객님소중한거래항상감사드립니다. 근로소득원천징수영수증(사업자인 STEP02. 2017-08_어김없이 STEP03. 2018-11_겨울햇살을 더보기 거창하고 변동성이후 G20정상회담에서 중국간의무역분쟁의 (중국정부 안정정책)을 비교해볼때 조정수준 없습니다그대가 결정되겠지

 0906. 판매량이 1164. 합니다나만의 바닥권 대답했습니다. 수령가능하십니다. 금리에서 -용혜원 지면으로나마 70%까지 뒷심도 전화드렸으며 완화시키고 초심에서 결심이 일하기 리브소개해 대방로지점에서 소종한 한주되세요혹시 종료)- 나은서비스를 있고햇살도 입출금거래내역 위안화등 감사드립니다작은 고객님황사가 무렵에 위치관련해서 중요한지 Xd결과는 의구심과 하남시 밑져야 타행자동이체로 79pt(0. 안부전화로 증권투자신탁(주식혼합-파생형)펀드가 어제내린 솔루션 지수연계 그네 아니었습니다. 기타재정운전자금대출 3년분할상환(3개월주기) 78DOW 짧게는 증시변동성이 길게는 드렸으며 EU와도 X수색전담직원 납부하게 29일소중한예금 중국외에 아닌것처럼 저녁되시고매우 안보이는 가고있네요. 드셨는지요?생일 인상될 11월14일 휘날리고 2017-11_짧아서 경제전망 사막의 주되시길 않을까. 안내문- 알이 되겠습니다행복한하루되세요 그후 드립니다지금 대표번호(1588-9008)로 기대. 깃을 메뉴에 한가로운 소중한분과 더편리하게 효창동지점 2018-07_최신정보 거래실적인정 오후시간되세요! 행복이죠그리고 재가입을 ELS판매가 시절에 보내세요!!. 되겠읍니다 차익실현매물 고객배려활동으로 리브(Liiv)출시!1. 주시거나FW 만물을 최초가격(226833원)의 보여드리는 연체중으로 추워졌지만 짬. 고객님떨어지는 유럽연합(EU) 금리우대쿠폰또한 황금연휴입니다. 최초가격(34700원)의 3372. KH-2호20150710 고객님행복가득한 분할매도 사람으로부터 더빠르게 아쉬움없는 완료되지 보내시고하시는 강경한 2017-09_넉넉한보름달처럼정이넘치는즐겁고풍요로운한가위명절보내세요 자라올라서. 보이지조차 유입되면서 매우동의부탁드립니다행복한 재무장관은 보이지는 예치하는 X(광고)당행에서 2738. 받아가세요!!! 98%)- 온도변화로 무겁습니다. 대구3공단종합금융센터지점담당자 대구3공단종합금융센터지점에서 건강하시길기원합니다. 봄!!행복 한계는 어릴 달러인덱스가 면제가 기대했지만 주도로 말씀드린대

 주말이네요향긋한 광주상공회의소(지하철 버려라오지도 2017-05_행복 2017-06_최선을 절정이라고 X은행화곡역지점 후면 용인흥덕지점 보내세요!!V()V설문조사시매우동의! 폐이지로 바랍니다즐겁고 푸른햇살처럼 세액공제(최대115. 농성역 하다이번에 되어전금융사 좋고이전보다 상관없습니다)? 되면사람들은 웃는다지보다 없으셨으면 맑아지고사람 하루되세요매우동의한다로칭찬해주세요 놈이라고뽐내면서 되면마음씨 좋겠습니다사랑의 보인다불쌍한 보내셨는지요?5월에는 고객님산은 펀드평가보고서 우편물수령방법을 X유성도안지점 고객님안녕하세요새로운 더할나위 X문정법조종합금융센터 새벽시간 2017-02_행복의 영등포중앙지점으로 임대차확인서신분증7. 2017-06_폭염에 더축 소비자신뢰지수 있거늘떨어진 마라바람이 2017-06_세상에 나쁜것이 보일뿐이다. 좋아보이고 어떤눈으로 살것이냐! 몫일뿐이다안녕하세요. 즐거운명절되세요 반복되더니3월도 감사드리고항상 노력하겠습니다(-)직원추천에 19041. 2018-08_달콤한 판교 감사드리고행복한 X고객님여행가실때 가족친지들과웃음꽃도 때문이야힘들고 생각하는너를 바라보는너무 반월산업단지지점을 2017-02_행복한금요일미소로인해더욱밝아지 X(홍천지점)직원번호 더치페이가입할 머니보내기(일일 2017-02_금요일은 난항 우리삶도눈부시게피어나길소망합니다. 2월19일에 옵션포함 초본1통 전화드립니다. X은행충무로역지점을 있도록전 X(모란역지점 한주보내세요!고객만족전화 금융서비스및 개설해 웃으면세상이 회신번호로 돕기 창구혼잡도가 납부마감일입니다. 되면약삭빠른 떤난다도움받을 되면세상이 보인다그냥 이대로가 요청한 -톨스토이 참여와 버티는 푸르른행복을 월간투자매거진 모기지파트너스(본점영업부)에서 뜻이다오늘 명절이었습니다. X하시기 바보이반중에서-세상이 10월19일 2017-06_행복 갑니다더불어 X용산종합금융센터 000이 2017-06_더워도 보유현황 때문에그를 31(화)(금리우대쿠폰은 신청대출상담시간 유의하시고편안하고 사랑하듯이산을 2017-12_추운날씨 드리고자우편

 활용도 강원도에 거주지 4월!바쁘게 강원보증재단 절기의 기대감으로기쁘게 (900억규모 부채통합대환 2017-06_매우만족(. 2(월) 옥련동지점이 짜증나기 소용돌이치기를 파도소리 2017-02_택시 거래소 2017-06_매우동의하시도록 9749전화주시거나 4647 1877-7308 송도역지점으로 들으면서 얘기입니다. 비시고 X은행(광고)-N. 대답할 라운지를찾아주시어 최저치로 겸손하고 솟는 목요일!!오늘 겨울바람 자동입금) 자동재예치(원금 감사드립니다활기찬 신용등급에도 X계장추천꼭 2018-06_날씨가 드리겟습니다. 듬뿍들어오도록 보네세요! 공정성균형호혜 수신(광고) 거울에 넘어가고있습니다. 결정합니다. 2017-04_촉촉히 테크노크리닉 2017-04_꾸준한거래항상감사드리며고객만족도조사매우동의를부탁드립니다 2018-08_지금 생각에는 2017-01_활짝 달러약세에 일어나면 X매직카대출)이 89%부터 2018년11월30일까지 비싸게 X성남하이테크밸리 2018-08_새로운8월 2017년도도절반이 바라며행복과 악화시키며 열어야신의 생활서비스를 블룸버그의 좋은날씨만큼 고발 될만큼 가입경로 럭셔리한 21414. 다녀가는 유의하시고늘 5월한달이 나누며입가에는 참으로행복합니다. 날이길 젖는다. 2017-09_고객님!항상 기회에는 고객님가을하늘이 와서산들거리는 야외그린가든 야외그린가든(양재동) 전과목만점도 내리기를 바랍니다고객님이 (1매당 펫팸족 초청이벤트 저희의 2018-08_활짝 연은총재 교환부스에서 X매직카대출!안녕하세요. 부른 X메디칼론을 운영하시는 받으셨는지요? 3262. 기죽지 25701. DC로 비상금 백영옥의 쿠폰번호-차량주차 목소리만 고객님건조한 8일은 하루되세요거래감사드립니다. 되시리라 남았지만 2018-08_고객님을 채워집니다. 바라며SMS입출금통지서비스가 별만을 주차비 기대해보면서 2018-02_음력 불가)-쿠폰번호 필수(타인 한주였던거 쿠폰번호가 확인하세요-당첨된 만원은 추구하면 가능합니다문의사항 생각하시기 입장팔찌 고객님안녕하세요봄향기 풍성하시기를

In [19]:
from soynlp.hangle import decompose

doublespace_pattern = re.compile('\s+')

def jamo_sentence(sent):

    def transform(char):
        if char == ' ':
            return char
        cjj = decompose(char)
        try:
            len(cjj)
        except:
            return ' '
        if len(cjj) == 1:
            return cjj
        cjj_ = ''.join(c if c != ' ' else '' for c in cjj)
        return cjj_

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_
'ㅇㅓ-ㅇㅣ-ㄱㅗ- ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-'
def jamo_sentence(sent):
    sent = sent.replace('XXX', 'X')
    def transform(char):
        if char == ' ':
            return char
        elif char == 'X':
            return ' X'
        elif char == '.':
            return '. '
        else:
            return char

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_

In [20]:
jamo_sentence(train_df.loc[0, 'text'])

' X은행성산 X팀장입니다. 행복한주말되세요'

In [21]:
train_df['jamo'] = train_df['text'].apply(lambda x: jamo_sentence(x))
test_df['jamo'] = test_df['text'].apply(lambda x: jamo_sentence(x))

In [22]:
# raw_corpus_fname = '\n'.join(pd.concat([train_df['jamo'], test_df['jamo']]))
# file=open('/content/textfile.txt','w')
# file.write(raw_corpus_fname)
# file.close()

In [27]:
# import fasttext
# model = fasttext.train_unsupervised('/content/textfile.txt', 
#                                     model='cbow' ,
#                                     loss = 'hs',        # hinge loss
#                                     ws=1,               # window size
#                                     lr = 0.01,          # learning rate
#                                     dim = 200,          # embedding dimension
#                                     epoch = 5,          # num of epochs
#                                     min_count = 10,     # minimum count of subwords
#                                     # encoding = 'utf-8', # input file encoding
#                                     thread = -1          # num of threads
#                                 )

In [ ]:
# model.save_model("/content/model_file.bin")
# model.save_model("/content/drive/My Drive/model_file.bin")